In [472]:
import requests
import pandas as pd

In [473]:
THE_FORGE = 10000002
JITA_TRADE_HUB = 60003760

def get_market_orders(region_id=THE_FORGE, location_id=JITA_TRADE_HUB, type_id=None, order_type=None):
    url = f"https://esi.evetech.net/latest/markets/{region_id}/orders/"

    params = {"datasource": "tranquility"}
    params["order_type"] = order_type if order_type else "all"
    if type_id:
        params["type_id"] = type_id
    if location_id:
        params["location_id"] = location_id

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print("Failed to fetch data")
        return None

    return pd.DataFrame(response.json())

In [474]:
def get_ids(item_names):
    url = "https://esi.evetech.net/latest/universe/ids/"
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=item_names)

    if response.status_code != 200:
        print("Failed to fetch item IDs")
        return None

    data = response.json()
    return {item["name"]: item["id"] for item in data.get("inventory_types", [])}

In [475]:
def get_item_volume(type_id):
    url = f"https://esi.evetech.net/latest/universe/types/{type_id}/"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch volume for type_id {type_id}")
        return None

    data = response.json()
    return data.get("volume")  # Extract volume field

In [476]:
COLUMNS = ["type_name", "is_buy_order", "volume_remain", "price", "volume", "price_per_volume"]

def get_jita_items_prices(item_ids):

    out = None

    for id in item_ids.values():
        df = get_market_orders(order_type="buy", type_id=id).sort_values(by="price", ascending=False).head(1)
        df["volume"] = get_item_volume(id)
        df["price_per_volume"] = (df["price"] / df["volume"]).astype(int)
        
        if out is None:            
            out = df
        else:  
            out = pd.concat([out, df])

    swapped_ids = {val: key for key, val in item_ids.items()}
    out["type_name"] = out["type_id"].map(swapped_ids)    

    return out[COLUMNS]

In [477]:
veldspar = [
    "Compressed Veldspar",
    "Compressed Concentrated Veldspar",
    "Compressed Dense Veldspar",
    "Compressed Stable Veldspar",
]

plagioclase = [
    "Compressed Plagioclase",
    "Compressed Azure Plagioclase",
    "Compressed Rich Plagioclase",
    "Compressed Sparkling Plagioclase",
]

scordite = [
    "Compressed Scordite",
    "Compressed Condensed Scordite",
    "Compressed Glossy Scordite",
    "Compressed Massive Scordite",
]

In [478]:
ids = get_ids(veldspar+plagioclase+scordite)
get_jita_items_prices(ids).sort_values(by="price_per_volume", ascending=False)

,type_name,is_buy_order,volume_remain,price,volume,price_per_volume
3,Compressed Glossy Scordite,True,1741853,45.83,0.0015,30553
21,Compressed Rich Plagioclase,True,7452918,75.43,0.0035,21551
12,Compressed Azure Plagioclase,True,12194015,73.55,0.0035,21014
34,Compressed Plagioclase,True,1828687,62.60,0.0035,17885
36,Compressed Massive Scordite,True,47939930,23.36,0.0015,15573
34,Compressed Condensed Scordite,True,4627494,22.89,0.0015,15260
35,Compressed Scordite,True,15276171,22.14,0.0015,14760
1,Compressed Stable Veldspar,True,999929,14.00,0.0010,14000
10,Compressed Sparkling Plagioclase,True,583398,47.16,0.0035,13474
23,Compressed Dense Veldspar,True,73465109,13.05,0.0010,13050
